In [1]:
import textract

text = textract.process("./文档3.docx")

In [ ]:
import re
import pandas as pd
def ItemExtraction(item,text):
    result={}
    text_pre=re.sub('\n','=',text)
    text_use=re.sub('\xc2\xa0','',text_pre) # remove space between item   
    Pattern='.*?'.join([item,'==(.*?)=='])
    matched_string=re.compile(Pattern).findall(text_use)
    n=0
    for value in matched_string:
        
        result.setdefault('matched',{}).update({item+'_'+str(n):value})
        n+=1
    result_pd=pd.DataFrame(result)
    return result_pd

In [131]:
ItemExtraction(item='性别',text=text)

,matched
性别_0,授权代理人：性别：民族：职业：身份证号码：工作单位：联系电话：邮政编码：住址：=委托人因鞍区...
性别_1,男
性别_2,男


In [130]:
def LabResultExtraction(title,ResultItem,text,time='报告日期'):
    result={}
    text_use=re.sub('\n','=',text)
        
    Pattern='.*?'.join([title,time,'(\d\d\d\d-\d\d-\d\d)',ResultItem,'==(.*?)=='])
    matched_string=re.compile(Pattern).findall(text_use)
    for t,value in matched_string:
        
        result.setdefault(ResultItem,{}).update({t:value})
    result_pd=pd.DataFrame(result)
    return result_pd

In [129]:
LabResultExtraction(ResultItem='淋巴细胞计数',text=text,title='检验科临检组检验报告单')

,淋巴细胞计数
2011-03-07,1.5
2011-03-08,0.7
2011-03-12,1.6
2011-03-13,2.1
2011-03-18,0.8
2011-03-20,0.2
